# Geographical Aggregation (Tourism)

> Geographical Hierarchical Forecasting on Australian Tourism Data

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

In this notebook we present an example on how to use `HierarchicalForecast` to produce coherent forecasts between geographical levels. We will use the classic Australian Domestic Tourism (`Tourism`) dataset, which contains monthly time series of the number of visitors to each state of Australia.

We will first load the `Tourism` data and produce base forecasts using an `AutoETS` model from `StatsForecast`, and then reconciliate the forecasts with several reconciliation algorithms from `HierarchicalForecast`. Finally, we show the performance is comparable with the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html) which uses the R package [fable](https://github.com/tidyverts/fable).

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast mlforecast datasetsforecast lightgbm sklearn

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df.head()

The dataset can be grouped in the following non-strictly hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
from hierarchicalforecast.utils import aggregate

In [ ]:
%%capture
Y_df, S_df, tags = aggregate(Y_df, spec)

In [ ]:
Y_df.head()

In [ ]:
S_df.iloc[:5, :5]

In [ ]:
tags['Country/Purpose']

### Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_train_df.groupby('unique_id').size()

## 2. Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `ETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [ ]:
%%capture
fcst = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], 
                     freq='QS', n_jobs=-1)
Y_hat_df_stats = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df_stats = fcst.forecast_fitted_values()

In [ ]:
%%capture
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingRegressor
from mlforecast.lag_transforms import ExpandingMean, RollingMean, ExpandingStd
from mlforecast.target_transforms import Differences
from mlforecast import MLForecast

In [ ]:
%%capture
mlf = MLForecast(
    models = {
        'lgbm': lgb.LGBMRegressor(verbosity=-1),
        'gbm':HistGradientBoostingRegressor()
    }, 
    freq='QS',
    target_transforms=[Differences([1, 4])],
    lags=[1, 2, 3, 4, 5, 6, 7, 8, 12],
    lag_transforms={  
        1: [ExpandingMean(), RollingMean(window_size=4)],
        4: [ExpandingMean(), RollingMean(window_size=2), RollingMean(window_size=4)],
        8: [RollingMean(window_size=4)]
    },
    date_features=['quarter', 'year']
)
mlf.fit(Y_train_df, fitted=True)
Y_hat_df_ml = mlf.predict(new_df=Y_train_df, h=8)
Y_fitted_df_ml = mlf.forecast_fitted_values()

In [ ]:
# TODO: Neuralforecast

In [ ]:
%%capture
Y_hat_df = Y_hat_df_stats.merge(Y_hat_df_ml, on=['unique_id', 'ds'])
Y_fitted_df = Y_fitted_df_stats.merge(Y_fitted_df_ml, on=['unique_id', 'ds', 'y'])

## 3. Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S=S_df, tags=tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df.head()

,unique_id,ds,AutoETS,lgbm,knn,gbm,AutoETS/BottomUp,lgbm/BottomUp,knn/BottomUp,gbm/BottomUp,AutoETS/MinTrace_method-mint_shrink,lgbm/MinTrace_method-mint_shrink,knn/MinTrace_method-mint_shrink,gbm/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,lgbm/MinTrace_method-ols,knn/MinTrace_method-ols,gbm/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,26523.543195,26500.318899,26720.376418,24381.672901,26945.553515,26760.608551,27030.626551,25397.376120,26894.104305,26712.629478,27006.091018,25894.419896,26641.438835,26563.968000,26792.200028
1,Australia,2016-04-01,24458.490282,24900.590405,24973.510958,24946.522366,22903.194016,26028.446233,26004.778564,26041.715196,23908.561820,25482.092771,25581.008937,25625.736968,24357.231461,25109.570985,25051.241552,25189.605116
2,Australia,2016-07-01,23974.055984,24139.018873,24438.339374,24554.762276,22411.401317,25345.135452,25406.127923,25400.038854,23401.977371,25040.504194,24942.231775,25292.196895,23865.928094,24451.714310,24562.536177,24822.730031
3,Australia,2016-10-01,24563.454495,26281.353485,26155.502665,26371.293563,23127.009693,26969.492032,27171.738862,26989.209978,24079.062804,26714.081669,26512.662099,26788.474306,24470.780870,26416.665730,26261.866112,26511.341898
4,Australia,2017-01-01,25990.068004,27341.272132,27533.213037,27455.293840,24518.047369,28893.386507,28735.701784,28974.810529,25531.236531,28131.721866,28052.346886,28257.463978,25901.383310,27592.439681,27696.836697,27692.431358


## 4. Evaluation 

The `HierarchicalForecast` package includes an `evaluate` function to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
from hierarchicalforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mase
from functools import partial

In [ ]:
eval_tags = {}
eval_tags['Total'] = tags['Country']
eval_tags['Purpose'] = tags['Country/Purpose']
eval_tags['State'] = tags['Country/State']
eval_tags['Regions'] = tags['Country/State/Region']
eval_tags['Bottom'] = tags['Country/State/Region/Purpose']

df = Y_rec_df.merge(Y_test_df, on=['unique_id', 'ds'])
evaluation = evaluate(df = df,
                      tags = eval_tags,
                      train_df = Y_train_df,
                      metrics = [rmse,
                                 partial(mase, seasonality=4)])

#evaluation.columns = ['level', 'metric', 'Base', 'BottomUp', 'MinTrace(mint_shrink)', 'MinTrace(ols)']
#TODO: Format columns in a programatic level
numeric_cols = evaluation.select_dtypes(include="number").columns
evaluation[numeric_cols] = evaluation[numeric_cols].map('{:.2f}'.format).astype(np.float64)

### RMSE

The following table shows the performance measured using RMSE across levels for each reconciliation method.

In [ ]:
%%capture
evaluation.query('metric == "rmse"')

### MASE


The following table shows the performance measured using MASE across levels for each reconciliation method.

In [ ]:
evaluation.query('metric == "mase"')

,level,metric,AutoETS,lgbm,knn,gbm,AutoETS/BottomUp,lgbm/BottomUp,knn/BottomUp,gbm/BottomUp,AutoETS/MinTrace_method-mint_shrink,lgbm/MinTrace_method-mint_shrink,knn/MinTrace_method-mint_shrink,gbm/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols,lgbm/MinTrace_method-ols,knn/MinTrace_method-ols,gbm/MinTrace_method-ols
1,Total,mase,1.59,0.40,0.29,0.31,3.16,1.32,1.46,1.45,2.07,0.72,0.67,0.87,1.67,0.32,0.30,0.40
3,Purpose,mase,1.32,1.12,1.04,1.07,2.28,1.48,1.79,1.55,1.49,1.06,1.16,1.12,1.25,0.88,0.97,0.90
5,State,mase,1.39,1.42,1.42,1.43,1.90,1.52,1.52,1.61,1.41,1.41,1.24,1.43,1.25,1.28,1.28,1.26
7,Regions,mase,1.12,1.21,1.40,1.22,1.19,1.24,1.46,1.27,1.01,1.19,1.32,1.21,0.99,1.15,1.39,1.16
9,Bottom,mase,0.98,1.29,1.66,1.34,0.98,1.29,1.66,1.34,0.94,1.26,1.61,1.31,1.01,1.32,1.78,1.34
11,Overall,mase,1.02,1.27,1.60,1.32,1.06,1.29,1.62,1.34,0.97,1.25,1.54,1.29,1.02,1.28,1.68,1.30


In [ ]:
# TODO: Put only the results that are in the table

### Comparison fable

Observe that we can recover the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html). The original results were calculated using the R package [fable](https://github.com/tidyverts/fable).

![Fable's reconciliation results](./imgs/AustralianDomesticTourism-results-fable.png)

### References
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)
- [Rob Hyndman, Alan Lee, Earo Wang, Shanika Wickramasuriya, and Maintainer Earo Wang (2021). "hts: Hierarchical and Grouped Time Series". URL https://CRAN.R-project.org/package=hts. R package version 0.3.1.](https://cran.r-project.org/web/packages/hts/index.html)
- [Mitchell O’Hara-Wild, Rob Hyndman, Earo Wang, Gabriel Caceres, Tim-Gunnar Hensel, and Timothy Hyndman (2021). "fable: Forecasting Models for Tidy Time Series". URL https://CRAN.R-project.org/package=fable. R package version 6.0.2.](https://CRAN.R-project.org/package=fable)